In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

Acknowledgement
https://www.kaggle.com/code/vbmokin/nlp-for-en-bert-cls-10-classifiers

In [3]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# models
from sklearn.linear_model import LinearRegression, Perceptron, RidgeClassifier, SGDClassifier, LassoCV
from sklearn.svm import SVC, LinearSVC, SVR
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier 
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier


# NN models
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

import torch
import transformers as ppb

import warnings
warnings.filterwarnings('ignore')


/Users/lpieri/mambaforge/envs/news-ml-exploratory/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_pickle('../../data/raw/raw.p')

In [5]:
df.head()

,title,description,published date,url,publisher,text,topic
0,Plight of homeless deepens as Turkey-Syria ear...,Plight of homeless deepens as Turkey-Syria ear...,"Thu, 09 Feb 2023 12:23:00 GMT",https://news.google.com/rss/articles/CBMiaGh0d...,"{'href': 'https://www.reuters.com', 'title': '...",Summary\n\nSummary Companies Death toll reache...,WORLD
1,Zelenskyy makes heartfelt call for Ukraine's E...,Zelenskyy makes heartfelt call for Ukraine's E...,"Thu, 09 Feb 2023 14:55:00 GMT",https://news.google.com/rss/articles/CBMiQGh0d...,"{'href': 'https://www.cnbc.com', 'title': 'CNBC'}",Ukrainian President Volodymyr Zelensky and Pre...,WORLD
2,Huge haul of cocaine floating at sea seized - CNN,Huge haul of cocaine floating at sea seized C...,"Thu, 09 Feb 2023 11:17:00 GMT",https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://www.cnn.com', 'title': 'CNN'}",CNN —\n\nMore than 3 tons of cocaine floating ...,WORLD
3,"5 things to know for February 9: Earthquake, C...","5 things to know for February 9: Earthquake, C...","Thu, 09 Feb 2023 11:53:00 GMT",https://news.google.com/rss/articles/CBMiSGh0d...,"{'href': 'https://www.cnn.com', 'title': 'CNN'}",CNN —\n\nGet '5 Things' in your inbox If your ...,WORLD
4,Kremlin Endorses Report on U.S. Involvement in...,Kremlin Endorses Report on U.S. Involvement in...,"Thu, 09 Feb 2023 11:36:32 GMT",https://news.google.com/rss/articles/CBMicmh0d...,"{'href': 'https://www.themoscowtimes.com', 'ti...",The Kremlin on Thursday endorsed a blog post b...,WORLD


# Category Distribution

In [6]:
df.topic.value_counts()

ENTERTAINMENT    490
NATION           470
TECHNOLOGY       454
BUSINESS         437
HEALTH           404
WORLD            391
SCIENCE          388
Name: topic, dtype: int64

Luckily a pretty even distribution especially given the nature of the scraper. The scraper gets denied on certain website.

# ML Task

Train a classifier for the categories.

In [7]:
df = df[['topic', 'text']]
# Manual OneHotEncoding because sklearns was taking me a bit.
for topic in df.topic.unique():
    df.loc[:,topic] = (df.topic == topic).astype(int)
    
    
df.drop(['topic'], axis=1, inplace=True)

In [8]:
df

,text,WORLD,NATION,BUSINESS,TECHNOLOGY,ENTERTAINMENT,SCIENCE,HEALTH
0,Summary\n\nSummary Companies Death toll reache...,1,0,0,0,0,0,0
1,Ukrainian President Volodymyr Zelensky and Pre...,1,0,0,0,0,0,0
2,CNN —\n\nMore than 3 tons of cocaine floating ...,1,0,0,0,0,0,0
3,CNN —\n\nGet '5 Things' in your inbox If your ...,1,0,0,0,0,0,0
4,The Kremlin on Thursday endorsed a blog post b...,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
3029,Listen Comment on this story Comment Gift Arti...,0,0,0,0,0,1,0
3030,NASA's Curiosity rover has found wave-rippled ...,0,0,0,0,0,1,0
3031,,0,0,0,0,0,1,0
3032,AUSTIN (KXAN) – A recent expedition to Patagon...,0,0,0,0,0,1,0


## BERT: Data preparing and modeling

In [9]:
# For pre-trained DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Other models: https://huggingface.co/transformers/pretrained_models.html

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
# Tokenization the sentences - break them up into word and subwords in the format BERT is comfortable with
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

Token indices sequence length is longer than the specified maximum sequence length for this model (1468 > 512). Running this sequence through the model will result in indexing errors


(3034, 19993)

In [11]:
# Creation variable to ignore (mask) the data padding
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)
attention_mask

(3034, 19993)


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:
# Modeling
input_ids = torch.tensor(padded).to(torch.int64)
attention_mask = torch.tensor(attention_mask).to(torch.int64)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
# Last hidden states
features = last_hidden_states[0][:,0,:].numpy()